# Propagate uncertainties with the `errors` add-on for CO2SYS-Matlab

James Orr<br>

<img align="left" width="50%" src="http://www.lsce.ipsl.fr/Css/img/banniere_LSCE_75.png"><br><br>

LSCE/IPSL, CEA-CNRS-UVSQ, Gif-sur-Yvette, France

27 February 2018 <br><br>
updated: 29 June 2020 

**Abstract**: This notebook shows you how to use the 'errors' add-on for CO2SYS-Matlab to propagate uncertainties. It uses CO2SYS-Matlab and the add-on routine `errors.m` (which itself calls another add-on routine `derivnum.m`) in `octave`, GNU's clone of Matlab. You can either inspect the HTML version of this file or execute its commands interactively in your browser. For the latter, you'll need to install `jupyter notebook`, `octave`, and `oct2py`, which includes the python-octave interface called `octavemagic`. Fortunately, that installation is very easy (see below).

#### Table of Contents:

    1. Basics (install & load octave)
    2. Propagate uncertainties: use `errors` add-on (with ALK-DIC input pair)
       * total uncertainty
       * uncertainty from the constants (standard input uncertainties)
       * uncertainty from the boron-to-salinity ratio (standard input uncertainty)
       * percent relative uncertainty
       * convert uncertainty in computed [H+] to uncertainty in computed pH

## 1. Basics

#### Run interactively

If you are visualizing this after clicking on the link to this file on github, you are seeing the HTML version of a jupyter notebook. Alternatively, you may run cells interactively and modify them if you have `jupyter notebook` installed on your machine.  To install that software, just download the anaconda open software installer for your computing platform (Windows, OS X, or Linux) from https://www.anaconda.com/ and then follow the easy install instructions at

https://docs.anaconda.com/anaconda/install/

Then just download this `jupyter notebook` file as well as the 3 routines in the src directory (`CO2SYS.m`, `errors.m`, and `derivnum.m`).  Afterwards, you'll only need to install `octave` and `oct2py` using the 1-line command in the following section.

#### Install `octavemagic`

#### To install the octavemagic funtionality, we must install `oct2py`, with the following command at the Unix prompt:

  `conda install -c conda-forge octave oct2py`
  
That command also installs `octave`. Then launch the notebook as usual with the following command:

  `jupyter notebook`

A new window or tab should then appear in your browser, showing the files in the directory from where you launched the above command. Then just click on one of the `.ipynb` files, such as this one.

Once the notebook file appears in your browser, move to any of its cells with your mouse. Run a cell by clicking on it and hitting Ctrl-Return.  Alternatively, type Shift-Return to run a cell and then move to the next one. More information on all the interactive commands is available in the Jupyter Notebook Quick Start Guide: http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/execute.html

At the top of the notebook, you'll see a number of Tabs (File, Edit, View, Insert, ...). Those tabs provide commands that will allow you to do whatever you like. Under the Help tab you'll find keyboard shortcuts for commands. Alternatively, a cheat sheet for short cuts to commands within `jupyter notebook` is available at https://zenodo.org/record/44973/files/ipynb-cheat-sheet.pdf . Or use the command palette after typing Ctrl-Shift-P.

#### Documentation for octavemagic

Details on using octavemagic are here: https://ipython.org/ipython-doc/2/config/extensions/octavemagic.html

#### Load octave magic function

Because octavemagic is now in conda's oct2py module, it is loaded with a slight modification to what is given on the above web page, i.e., now with the command below

In [1]:
%load_ext oct2py.ipython

#### Specify the directory where you have put the Matlab routines `CO2SYS.m`, `errors.m`, and `derivnum.m`.

In [2]:
%%octave
addpath ("~/Software/MATLAB/CO2SYS-MATLAB/src")

## 2. Propagate uncertainties with new `errors` add-on for CO2SYS-Matlab

In [3]:
%%octave
help errors

'errors' is a function from the file /homel/orr/Software/MATLAB/CO2SYS-MATLAB/src/errors.m

 errors()

 This subroutine propagates uncertainties for the marine carbonate chemistry calculations

 from errors (or uncertainties) on six input

  - pair of carbonate system variables

...

available in the online version of the manual.  Use the command

'doc <topic>' to search the manual index.

Help and information about Octave is also available on the WWW

at http://www.octave.org and via the help@octave.org

mailing list.

### 2.1 Specify input variables and choices

In [4]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 2300;        % ALK
    PAR2 = 2000;        % DIC
    PAR1TYPE = 1;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 2;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 25;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

    # Input Parameters:
    pHSCALEIN = 2;      % pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 15; % set for K1 & K2: (a) 10=Lueker et al. (2000); (b) 14=Millero (2010)
    KSO4CONSTANTS = 1;  % KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

# Input for CO2SYS:
# --------------------------
    # Input variables for error propagation:
    r = 0.0;            % Correlation between **uncertainties** in PAR1 and PAR2 (-1 < r < 1)
    ePAR1 = 2;          % uncertainty in PAR1 (same units as PAR1)
    ePAR2 = 2;          % uncertainty in PAR2 (same units as PAR2)
    eSAL = 0;           % uncertainty in Salinity
    eTEMP = 0;          % uncertainty in Temperature
    eSI = 4;            % uncertainty in Sit
    ePO4 = 0.1;         % uncertainty in Pt
    #Default uncertainties (pK units): epK0 , epK1, epK2, epKb, epKw, epKspA, epKspC
    epK =                [0.004, 0.015, 0.03, 0.01, 0.01,   0.02,  0.02];
    #Default uncertainty for Total boron (0.01, i.e., a 1% relative uncertainty)
    eBt = 0.01;


### 2.2 Propagate uncertainties neglecting uncertainties from equilibrium constants & total boron

In [5]:
%%octave
% With no errors from Ks
    epK = 0.0;
    eBt = 0.0;
    [e, ehead, enice] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4,...
                              ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...,
                              pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);
#   Print results
#    e
#    ehead
#    enice

# Print (nicely formatted):
  printf("%s   %s %s %s %s %s %s %s %s \n", ehead{1:9});
  printf("%f %f  %f  %f  %f %f %f     %f     %f \n", e(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

0.077469 3.805834  3.792600  3.408018  1.848113 0.130039 0.044126     0.028534     3.883373

### 2.3 Propagate uncertainties sequentially accounting for uncertainties from constants & total boron 

#### Default uncertainties in equilibrium constants, but assume no uncertainty in total boron $B_\text{T}$

In [6]:
%%octave
% With std errors from constants except for Bt
    epK = [0.002, 0.0075, 0.015, 0.01, 0.01, 0.02, 0.02];
    eBt = 0.0;
    [ek, ekhead, eknice] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

0.189647 9.461716 9.428816 4.261992 3.008619 0.319733  0.238120     0.153982     9.654489

#### Default uncertainties in equilibrium constants & default uncertainty in total $B_\text{T}$ (1%)

In [7]:
%%octave
% With std uncertainties from constants & Bt
    epK = [0.002, 0.0075, 0.015, 0.01, 0.01, 0.02, 0.02];
    eBt = 0.02;
    [ekb, ekbhead] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

0.195943 9.731243 9.697405 4.425336 3.265272 0.329042  0.240039     0.155223     9.929506

Same calculation, but with a simpler way to specify the defaults for epK and eBt

In [8]:
%%octave
% With std uncertainties from constants & Bt
    epK = '';
    eBt = '';
    [ekb, ekbhead] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));

u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

0.195943 9.731243 9.697405 4.425336 3.265272 0.329042  0.240039     0.155223     9.929506

### 2.4 Summarize effects of accouning for uncertainties in equil. constants & total boron

#### Fractional increase in propagated uncertainty due to accounting for uncertainties from equilibrium constants (default values)

In [9]:
%%octave
  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", e(1:9));
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9) ./ e(1:9));


u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

2.448047 2.486109 2.486109 1.250578 1.627941 2.458746  5.396363     5.396363     2.486109

Conclusion: Accounting for uncertainties from the equilibrium constants increases propagated uncertainties by 1.3 to 5.4 times for the At-Ct pair with default uncertainties.


#### Fractional increase in propagated uncertainty due to accounting for uncertainty in total boron (default value)

In [10]:
%%octave
  printf("%s   %s %s %s %s %s %s %s %s \n", ekbhead{1:9});
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ek(1:9));
 %printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9));
  printf("%f %f %f %f %f %f  %f     %f     %f \n", ekb(1:9) ./ ek(1:9));


u(Hin)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in) u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

1.033198 1.028486 1.028486 1.038326 1.085306 1.029115  1.008062     1.008062     1.028486

Conclusion: Accounting for uncertainty in total boron increases propagated absolute uncertainties by about 3% for most computed variables (except 9% for $[\mathrm{CO}_3^{2-}]$) when using the default uncertainty (eBt = 0.02) but by less than 1% when using the eBt=0.01, i.e., with the At-Ct pair.


### 2.5 Compute percent relative uncertainties 

#### Compute CO2SYS variables (the reference)

In [11]:
%%octave
% With std uncertainties from Ks
    [d, dhead, dnice] = CO2SYS (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

#### Reorder CO2SYS output in new array having same order as output array from `errors` (above, section 2.3) for later division (below)

In [12]:
%%octave
ALK     = d(:,1);    %01 - TAlk                 (umol/kgSW)
DIC     = d(:,2);    %02 - TCO2                 (umol/kgSW)
pH      = d(:,3);    %03 - pHin                 ()
pCO2    = d(:,4);    %04 - pCO2 input           (uatm)
fCO2    = d(:,5);    %05 - fCO2 input           (uatm)
HCO3    = d(:,6);    %06 - HCO3 input           (umol/kgSW)
CO3     = d(:,7);    %07 - CO3 input            (umol/kgSW)
CO2     = d(:,8);    %08 - CO2 input            (umol/kgSW)
Hfree   = d(:,13);   %13 - Hfree input          (umol/kgSW)
OmegaCa = d(:,15);   %15 - OmegaCa input        ()
OmegaAr = d(:,16);   %16 - OmegaAr input        ()
xCO2    = d(:,17);   %17 - xCO2 input           (ppm)

H = 10**(-pH) * 1e9; # Convert pH to H+ and then from mol/kg to nmol/kg
dar = [H, pCO2, fCO2, HCO3, CO3, CO2, OmegaCa, OmegaAr, xCO2];
%dar

#### Compute percent error

In [13]:
%%octave
  perr = 100* ekb(1:9) ./ dar;

# Show uncertainties in percent of base value computed with CO2SYS
  printf("%s  %s   %s   %s  %s    %s   %s %s %s \n", ehead{1:9});
  printf("%4.2f  %6.2f     %7.2f    %8.2f    %7.2f     %7.2f    %7.2f      %7.2f      %7.2f \n", perr(1:9));

u(Hin)  u(pCO2in)   u(fCO2in)   u(HCO3in)  u(CO3in)    u(CO2in)   u(OmegaCAin) u(OmegaARin) u(xCO2in)

2.71    3.20        3.20        0.25       1.58        3.17       4.87         4.87         3.20

#### Compute absolute error of pH from relative error in H

An absolute change in pH is equivalent to a relative change in H+.  That is, it can be shown that for **small** changes in H+ (dH):

\begin{equation}
d\text{pH} = -\frac{1}{\text{ln}(10)}  \frac{d\text{H}}{\text{H}}
\end{equation}

To get to this result, 

1) start with the basic definition: $\text{pH} = - \text{log}_{10} \, [\text{H}^+]$,

2) convert the logarithm base 10 to a natural log with $\text{log}_{10}(x) = \text{ln}(x) / \text{ln}(10)$, 

3) take the derivative of each side, and

4) plug in the definition of the derivative of a natural log, i.e.,  $d \text{ln}(x) = dx/x$.

##### Do the actual calculation

In [14]:
%%octave
# We drop the minus sign below because uncertainties are positive by definition (1 sigma)
# Note that "(0.01 * perr)" is the fractional error in hydrogen ion concentration (i.e., dH/H)
  dpH = (0.01 * perr(1))/log(10);
  dpH

dpH =  0.011768

### 2.6 Example of using `errors.m` routine with the pH-At pair

In [15]:
%%octave

# Standard input for CO2SYS:
# --------------------------
    # Input Variables:
    PAR1 = 2300;        % ALK
    PAR2 = 8.1;         % pH
    PAR1TYPE = 1;       % 1=ALK, 2=DIC, 3=pH, 4=pCO2, 5=fCO2
    PAR2TYPE = 3;       % Same 5 choices as PAR1TYPE
    SAL = 35;           % Salinity
    TEMPIN = 18;        % Temperature (input)
    TEMPOUT = 18;       % Temperature (output)
    PRESIN = 0;         % Pressure (input)
    PRESOUT = PRESIN;   % Pressure (output)
    SI = 60;            % Total dissolved inorganic silicon (Sit)
    PO4 = 2;            % Total dissoloved inorganic Phosphorus (Pt)

# Input Parameters:
    pHSCALEIN = 1;      %% pH scale (1=total, 2=seawater, 3=NBS, 4=Free)
    K1K2CONSTANTS = 10; %% set for K1 & K2: (a) 10=Lueker et al. (2000); (b) 14=Millero (2010)
    KSO4CONSTANTS = 1;  %% KSO4 of Dickson (1990a) & Total dissolved boron (Bt) from Uppstrom (1974)

# Input for CO2SYS:
# --------------------------
    # Input variables for error propagation:
    r = 0.0;            % Correlation between **uncertainties** in PAR1 and PAR2 (-1 < r < 1)
    ePAR1 = 2;          % uncertainty in PAR1 (same units as PAR1)
    ePAR2 = 0.01;          % uncertainty in PAR2 (same units as PAR2)
    eSAL = 0;           % uncertainty in Salinity
    eTEMP = 0;          % uncertainty in Temperature
    eSI = 4;            % uncertainty in Sit
    ePO4 = 0.1;         % uncertainty in Pt
    #Default uncertainties: epK0 , epK1, epK2, epKb, epKw, epKspA, epKspC
    epK = [0.002, 0.0075, 0.015, 0.01, 0.01, 0.02, 0.02];
    #Default uncertainty for Total Boron: 0.01 (i.e., a 1% relative error)
    eBt = 0.02;

In [16]:
%%octave
% With std uncertainties from constants & Bt
    epK = '';
    eBt = '';
    [e, ehead, eunits] = errors (PAR1, PAR2, PAR1TYPE, PAR2TYPE, SAL, TEMPIN, TEMPOUT, PRESIN, PRESOUT, SI, PO4, ...
                             ePAR1, ePAR2, eSAL, eTEMP, eSI, ePO4, epK, eBt, r, ...
                             pHSCALEIN, K1K2CONSTANTS, KSO4CONSTANTS);

  printf("%s   %s %s %s %s  %s %s %s %s \n", ehead{1:9});
  printf("%s   %s  %s    %s %s %s    %s      %s          %s \n", eunits{1:9});
  printf("%f  %f  %f  %f  %f %f  %f     %f     %f \n", e(1:9));

u(TCO2)   u(pCO2in) u(fCO2in) u(HCO3in) u(CO3in)  u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

(umol/kg)   (uatm)  (uatm)    (umol/kg) (umol/kg) (umol/kg)    ( )      ( )          (ppm)

8.327959  11.585334  11.545050  14.191636  6.483093 0.392120  0.261038     0.168802     11.821373

### 2.7 Example of using `errors.m` routine with the pH-pCO2 pair

In [17]:
%%octave

PAR1 = 7.91;    % pH_T
PAR2 = 500;    % pCO2
PAR1TYPE = 3;
PAR2TYPE = 4;
SAL = 35;
TEMPIN = 18;
TEMPOUT = 18;
PRESIN = 0;
PRESOUT = PRESIN;
SI = 0;
PO4 = 0;
pHSCALEIN = 1;   % total scale
K1K2CONSTANTS = 10; % Lueker 2000
KSO4CONSTANTS = 1;  % KSO4 of Dickson & TB of Uppstrom 1979
ePAR1 = 0.02;    % pH
ePAR2 = 5;    % pCO2
eSAL = 0;
eTEMP = 0;
eSI = 0;
ePO4 = 0;

% Set input uncertainties for constants and Bt/S to zero 
epK=0;
eBt=0;

r=0;

[e, ehead, eunits] = errors (PAR1,PAR2,PAR1TYPE,PAR2TYPE,SAL,TEMPIN,TEMPOUT,PRESIN,PRESOUT,SI,PO4,...
ePAR1,ePAR2,eSAL,eTEMP,eSI,ePO4,epK,eBt,r,pHSCALEIN,K1K2CONSTANTS,KSO4CONSTANTS);

  printf("%s   %s %s %s %s  %s %s %s %s \n", ehead{1:9});
  printf("%s   %s  %s    %s %s %s    %s      %s          %s \n", eunits{1:9});
  printf("%f  %f  %f  %f  %f %f  %f     %f     %f \n", e(1:9));

u(TAlk)   u(TCO2) u(fCO2in) u(HCO3in) u(CO3in)  u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

(umol/kg)   (umol/kg)  (uatm)    (umol/kg) (umol/kg) (umol/kg)    ( )      ( )          (ppm)

104.239312  91.153989  4.982614  80.460830  10.711577 0.170842  0.255752     0.165384     5.101870

In [18]:
%%octave

# Use default uncertainties for constants instead of setting them to zero (case in previous cell)
epK = '';
eBt = '';

[e, ehead, eunits] = errors (PAR1,PAR2,PAR1TYPE,PAR2TYPE,SAL,TEMPIN,TEMPOUT,PRESIN,PRESOUT,SI,PO4,...
ePAR1,ePAR2,eSAL,eTEMP,eSI,ePO4,epK,eBt,r,pHSCALEIN,K1K2CONSTANTS,KSO4CONSTANTS);

  printf("%s   %s %s %s %s  %s %s %s %s \n", ehead{1:9});
  printf("%s   %s  %s    %s %s %s    %s      %s          %s \n", eunits{1:9});
  printf("%f  %f  %f  %f  %f %f  %f     %f     %f \n", e(1:9));

u(TAlk)   u(TCO2) u(fCO2in) u(HCO3in) u(CO3in)  u(CO2in) u(OmegaCAin) u(OmegaARin) u(xCO2in)

(umol/kg)   (umol/kg)  (uatm)    (umol/kg) (umol/kg) (umol/kg)    ( )      ( )          (ppm)

110.150090  96.891395  4.982614  86.053315  11.617020 0.188087  0.305116     0.197306     5.101870

In [19]:
%%octave
ehead, eunits

ehead =

{

  [1,1] = u(TAlk)

  [2,1] = u(TCO2)

  [3,1] = u(fCO2in)

...

  [15,1] = (umol/kg)

  [16,1] = ( )

  [17,1] = ( )

  [18,1] = (ppm)

}